In [2]:
import os
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Coron3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from glob import glob
import astropy.io.fits as fits
import numpy as np
import time
import pandas as pd
import json
import pickle
from json import JSONEncoder
from collections import defaultdict
from CoronPipeline import MyCoron3Pipeline

In [3]:
os.environ["CRDS_PATH"] = '/home/bariskurtkaya/crds_cache/jwst_ops'
os.environ["CRDS_SERVER_URL"] = 'https://jwst-crds.stsci.edu'

In [4]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [20,15]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

In [5]:
import jwst
print(jwst.__version__)

1.9.5.dev3+g22acc46


In [6]:
PROPOSAL_ID = '1441'
INSTRUME = 'NIRCAM'
#DETECTOR = 'nrca2'

# Pipeline things



<div class="alert alert-block alert-info">
Now we have the fits files. We will create our coronagrahic images using the JWST pipeline. Our data has the suffix <b>.rateints</b> just like we needed. 
While processing coronagraphy data, the common procedure is using the following pipeline:
    <ul> 
        <li> calwebb_detector1 $\rightarrow$ <strong>from jwst.pipeline import Detector1Pipeline</strong>
        <li> calwebb_image2  $\rightarrow$ <strong>from jwst.pipeline import Image2Pipeline</strong>
        <li> calwebb_coron3  $\rightarrow$ <strong>from jwst.pipeline import Coron3Pipeline</strong>
    </ul>
</div>

We have the outputs of the first stage. They all have the `rateints.fits` suffix. By passing them through the second pipeline we are going to end up with `calints.fits` files which is a last step before getting the coron products.

In [7]:
# Path containing the rateints data
directory = f'/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/{INSTRUME}/{PROPOSAL_ID}/mastDownload/JWST/'

In [8]:
directory

'/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/'

### Stage2

In [9]:
def get_exptypes(fits_):
    
    for file in range(len(fits_)):
        
        f = fits.open(fits_[file])
        exp_type = f[0].header['EXP_TYPE']
        print(exp_type)

In [10]:
def runimg2(filename, output_dir):
    
    img2 = Image2Pipeline()
    img2.output_dir = output_dir
    img2.save_results = True
    img2(filename)

In [11]:
def delete_files(files):
    for file in files:

        if os.path.isfile(file):
            os.remove(file)
            print("File has been deleted")
        else:
            print("File does not exist")

In [11]:
rateints_files = glob(os.path.join(directory, '**/*nrcalong_rateints.fits'))

In [12]:
batch_size = 4

In [13]:
for i in range(0,len(rateints_files),batch_size):
    
    for f in rateints_files[i:i+batch_size]:
        
        
        output_dir = '/'.join(f.split('/')[:-1]) + '/' 
        runimg2(f,output_dir)
    time.sleep(1)

2023-02-21 02:04:24,703 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2023-02-21 02:04:24,706 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2023-02-21 02:04:24,710 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2023-02-21 02:04:24,712 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2023-02-21 02:04:24,714 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2023-02-21 02:04:24,717 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2023-02-21 02:04:24,835 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441001001_02102_00001_nrcalong/jw01441001001_02102_00001_nrcalong_rateints.fits',).
2023-02-21 02:04:24,841 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file':

In [14]:
calints_data = glob(os.path.join(directory, '**/*nrcalong_calints.fits'))

In [15]:
calints1 = fits.open(calints_data[0])
calints1.info()

Filename: /home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441001001_02102_00001_nrcalong/jw01441001001_02102_00001_nrcalong_calints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     347   ()      
  1  SCI           1 ImageHDU        81   (64, 64, 1)   float32   
  2  ERR           1 ImageHDU        11   (64, 64, 1)   float32   
  3  DQ            1 ImageHDU        12   (64, 64, 1)   int32 (rescales to uint32)   
  4  INT_TIMES     1 BinTableHDU     24   1R x 7C   [J, D, D, D, D, D, D]   
  5  VAR_POISSON    1 ImageHDU        10   (64, 64, 1)   float32   
  6  VAR_RNOISE    1 ImageHDU        10   (64, 64, 1)   float32   
  7  VAR_FLAT      1 ImageHDU        10   (64, 64, 1)   float32   
  8  ASDF          1 BinTableHDU     11   1R x 1C   [25057B]   


### Stage 3

The stage 3 is kinda different than the other steps. To run this step we need an association (ASN) file. Thus, we need some functions to create ASN files.

In [16]:
len(calints_data)

90

In [17]:
from jwst.associations.mkpool import mkpool
from jwst.associations.lib.rules_level3 import Asn_Lv3Coron
from jwst.associations import AssociationPool, AssociationRegistry
from jwst.associations.mkpool import from_cmdline, mkpool
from jwst.associations import generate
from astropy.table import Table

In [18]:
pool = mkpool(calints_data)

In [19]:
pool_df = pool.to_pandas()
pool_df.to_csv(f'calints_{INSTRUME}_{PROPOSAL_ID}_pool.csv',index=False)

In [20]:
def t_path(partial_path):
    __file__ = '/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/notebooks/asn_coron_rule.py'
    test_dir = os.path.dirname(__file__)
    return os.path.join(test_dir, partial_path)

In [21]:
registry = AssociationRegistry([t_path('asn_coron_rule.py')],
                              include_default=False)

In [22]:
asns = generate(pool,registry)

202302210208:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441001001_03106_00001_nrcalong/jw01441001001_03106_00001_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202302210208:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441002001_03106_00001_nrcalong/jw01441002001_03106_00001_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202302210208:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441003001_03106_00006_nrcalong/jw01441003001_03106_00006_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202302210208:WARNING:jwst.associations.l

In [23]:
len(asns)

3

In [24]:
asns

[{
     "asn_type": "coron3",
     "asn_rule": "Asn_Lv3Coron",
     "version_id": null,
     "code_version": "1.9.5.dev3+g22acc46",
     "degraded_status": "No known degraded exposures in association.",
     "program": "01441",
     "constraints": "DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '01441'})\nDMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'NIRCAM'})\nDMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'F250M'})\nDMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'MASKBAR'})\nDMSAttrConstraint({'name': 'opt_elem3', 'sources': ['fxd_slit'], 'value': None})\nDMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'SUB320ALWB'})\nDMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'NRC_CORON'})\nDMSAttrConstraint({'name': 'target', 'sources': ['targetid'], 'value': '3'})\nDMSAttrConstraint({'name': 'bkgdtarg', 'sources': ['bkgdtarg'], 'value': None})

In [25]:
class FileEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [26]:
def get_asns(asns):
    
    asn_list = []
    for idx,asn in enumerate(asns):
        asn_dict = {}
        
        for i,j in zip(asn.keys(),asn.values()):
            asn_dict[i] = j
            
        asn_list.append(asn_dict)
        
        
        # I couldn't figure out how to serialize the abc.AsnCoron_Lv3 object and save it as a json file
        # For a temporary solution we will have a list that contains asn files as dictionaries
        # In this way, we can easily run the pipeline
        '''
        FileEncoder().encode(asn_dict)
        with open(f'asnfile_{idx}.json','w') as output:
            encodedJSON = json.dump(asn_dict,output,indent=4,cls=FileEncoder)
        '''
        
    return asn_list
            

In [27]:
def runcoron(filename, output_dir):

    coron = MyCoron3Pipeline()
    coron.output_dir = output_dir
    coron.save_results = True
    coron.process(filename)

In [28]:
def get_stage3_products(asns,directory):
    
    for t in range(len(asns)):

        asn_dict = {}
        for i,j in zip(asns[t].keys(),asns[t].values()):
            asn_dict[i] = j
            
        runcoron(asn_dict,directory)
        
        

In [29]:
print(len(asns[0]['products'][0]['members']))
print(len(asns[1]['products'][0]['members']))
#print(len(asns[2]['products'][0]['members']))

17
17


In [30]:
get_stage3_products(asns,directory)

2023-02-21 02:09:01,620 - stpipe.MyCoron3Pipeline - INFO - MyCoron3Pipeline instance created.
2023-02-21 02:09:01,623 - stpipe.MyCoron3Pipeline.stack_refs - INFO - StackRefsStep instance created.
2023-02-21 02:09:01,625 - stpipe.MyCoron3Pipeline.align_refs - INFO - AlignRefsStep instance created.
2023-02-21 02:09:01,628 - stpipe.MyCoron3Pipeline.klip - INFO - KlipStep instance created.
2023-02-21 02:09:01,632 - stpipe.MyCoron3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-02-21 02:09:01,635 - stpipe.MyCoron3Pipeline.resample - INFO - ResampleStep instance created.
2023-02-21 02:09:01,636 - stpipe.MyCoron3Pipeline - INFO - Starting calwebb_coron3 ...
2023-02-21 02:09:05,060 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01441007001_03108_00002_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2023-02-21 02:09:05,088 - CRDS - INFO -  Fetching  /home/bariskurtkaya/crds_cache/jwst_ops/references/jwst/nircam/jwst_nirca

Output: jw01441-a3001_t003_nircam_f250m-maskbar-sub320alwb
Prod: {'name': 'jw01441-a3001_t003_nircam_f250m-maskbar-sub320alwb', 'members': [{'expname': '/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441005001_03108_00001_nrcalong/jw01441005001_03108_00001_nrcalong_calints.fits', 'exptype': 'science', 'exposerr': None, 'asn_candidate': "[('o005', 'observation')]"}, {'expname': '/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441006001_03108_00001_nrcalong/jw01441006001_03108_00001_nrcalong_calints.fits', 'exptype': 'science', 'exposerr': None, 'asn_candidate': "[('o006', 'observation')]"}, {'expname': '/home/bariskurtkaya/github/AI-Boosted-Coronagraphy/dataset/NIRCAM/1441/mastDownload/JWST/jw01441007001_03108_00002_nrcalong/jw01441007001_03108_00002_nrcalong_calints.fits', 'exptype': 'psf', 'exposerr': None, 'asn_candidate': "[('o007', 'observation')]"}, {'expname': '/home/bariskurtkaya/github/AI

2023-02-21 02:09:05,274 - stpipe.MyCoron3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/home/bariskurtkaya/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2023-02-21 02:09:05,275 - stpipe.MyCoron3Pipeline - INFO - Prefetch for PSFMASK reference file is '/home/bariskurtkaya/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_psfmask_0042.fits'.
2023-02-21 02:09:05,689 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01441007001_03108_00001_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2023-02-21 02:09:05,693 - stpipe.MyCoron3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/home/bariskurtkaya/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2023-02-21 02:09:05,694 - stpipe.MyCoron3Pipeline - INFO - Prefetch for PSFMASK reference file is '/home/bariskurtkaya/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_psfmask_0042.fits'.
2023-02-21 02:09:05,884 - stpipe.My

KeyboardInterrupt: 

In [ ]:
print('Finished')